### Imports

In [1]:
# Standard library imports
import os
import re
import unicodedata
import warnings

# Third-party library imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from matplotlib.ticker import PercentFormatter
import seaborn as sns
import textstat
from textblob import TextBlob
from textblob_nl import PatternAnalyzer
from tqdm.notebook import tqdm
import requests


# 1. Consumentenvertrouwen, economisch klimaat en koopbereidheid: ongecorrigeerd

https://opendata.cbs.nl/#/CBS/nl/dataset/83694NED/table?searchKeywords=%E2%80%93%20consumentenvertrouwen,%20economisch%20klimaat%20en%20koopbereidheid

1.1 Data inladen/inzien


In [12]:
import pandas as pd

# 1. Data inladen
df_raw = pd.read_csv(
    r"E:\Nieuw dataset\Consumentenvertrouwen, economisch klimaat en koopbereidheid ongecorrigeerd\Consumentenvertrouwen__economisch_klimaat_en_koopbereidheid__gecorrigeerd_27042025_161526.csv",
    sep=";"
)

display(df_raw)


,Onderwerp,Unnamed: 1,1986 april,1986 mei,1986 juni,1986 juli,1986 augustus,1986 september,1986 oktober,1986 november,...,2024 juli,2024 augustus,2024 september,2024 oktober,2024 november,2024 december,2025 januari,2025 februari,2025 maart,2025 april
0,Consumentenvertrouwen,NaN,2,8,15,20,21,21,21,21,...,-24,-24,-21,-22,-25,-26,-28,-32,-34,-37
1,Economisch klimaat,NaN,6,17,33,45,47,45,45,44,...,-39,-42,-37,-37,-43,-45,-50,-54,-57,-61
2,Koopbereidheid,NaN,0,1,3,4,4,5,6,5,...,-14,-13,-11,-12,-12,-13,-14,-17,-18,-20


data opmgooien

In [ ]:


# 2. Transponeren
df_transposed = df_raw.set_index('Onderwerp').T

# 3. Index hernoemen naar 'Periode'
df_transposed.index.name = 'Periode'
df_transposed.reset_index(inplace=True)

# 4. Displayen
display(df_transposed)


Onderwerp,Periode,Consumentenvertrouwen,Economisch klimaat,Koopbereidheid
0,Unnamed: 1,NaN,NaN,NaN
1,1986 april,2.0,6.0,0.0
2,1986 mei,8.0,17.0,1.0
3,1986 juni,15.0,33.0,3.0
4,1986 juli,20.0,45.0,4.0
...,...,...,...,...
465,2024 december,-26.0,-45.0,-13.0
466,2025 januari,-28.0,-50.0,-14.0
467,2025 februari,-32.0,-54.0,-17.0
468,2025 maart,-34.0,-57.0,-18.0


In [ ]:


# 5. Periode omzetten naar datetime
df_transposed['Periode'] = pd.to_datetime(df_transposed['Periode'], format='%Y %B', errors='coerce')

# 6. Omzetten naar YYYY/MM string
df_transposed['Periode'] = df_transposed['Periode'].dt.strftime('%Y/%m')

# 7. Filteren op periode 2015-01 t/m 2025-04
df_transposed = df_transposed[
    (df_transposed['Periode'] >= '2015/01') & 
    (df_transposed['Periode'] <= '2025/04')
]

# 8. Kolommen herschikken (alleen als ze bestaan)
kolommen = ['Periode']
for kolom in ['Consumentenvertrouwen', 'Economisch klimaat', 'Koopbereidheid']:
    if kolom in df_transposed.columns:
        kolommen.append(kolom)

df_transposed = df_transposed[kolommen]

# 9. Resultaat bekijken
display(df_transposed)

Onderwerp,Periode,Consumentenvertrouwen,Economisch klimaat,Koopbereidheid
349,2015/04,10.0,30.0,-4.0
354,2015/09,11.0,28.0,0.0
356,2015/11,14.0,31.0,3.0
357,2015/12,13.0,27.0,4.0
361,2016/04,6.0,12.0,1.0
366,2016/09,12.0,26.0,3.0
368,2016/11,20.0,39.0,8.0
369,2016/12,21.0,39.0,8.0
373,2017/04,26.0,50.0,10.0
378,2017/09,23.0,44.0,10.0


In [24]:
# ---- Optie 2: Interpolatie toevoegen ----

# Genereer volledige maandlijst
maanden_volledig = pd.date_range(start='2015-01-01', end='2025-04-01', freq='MS').strftime('%Y/%m')
df_maanden = pd.DataFrame({'Periode': maanden_volledig})

# Merge volledige maanden met de transposed data
df_consumentenvertrouwen_interpolatie = df_maanden.merge(df_transposed, on='Periode', how='left')

# Interpoleren (lineair) op de relevante kolommen
df_consumentenvertrouwen_interpolatie[['Consumentenvertrouwen', 'Economisch klimaat', 'Koopbereidheid']] = \
    df_consumentenvertrouwen_interpolatie[['Consumentenvertrouwen', 'Economisch klimaat', 'Koopbereidheid']].interpolate(method='linear')

# 1. Eerst interpoleren
df_consumentenvertrouwen_interpolatie[['Consumentenvertrouwen', 'Economisch klimaat', 'Koopbereidheid']] = \
    df_consumentenvertrouwen_interpolatie[['Consumentenvertrouwen', 'Economisch klimaat', 'Koopbereidheid']].interpolate(method='linear')

# 2. Daarna backward fill voor beginwaarden
df_consumentenvertrouwen_interpolatie[['Consumentenvertrouwen', 'Economisch klimaat', 'Koopbereidheid']] = \
    df_consumentenvertrouwen_interpolatie[['Consumentenvertrouwen', 'Economisch klimaat', 'Koopbereidheid']].fillna(method='bfill')

# Resultaat bekijken
display(df_consumentenvertrouwen_interpolatie)

C:\Users\dylan\AppData\Local\Temp\ipykernel_20464\1763514700.py:20: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_consumentenvertrouwen_interpolatie[['Consumentenvertrouwen', 'Economisch klimaat', 'Koopbereidheid']].fillna(method='bfill')


,Periode,Consumentenvertrouwen,Economisch klimaat,Koopbereidheid
0,2015/01,10.00,30.0,-4.00
1,2015/02,10.00,30.0,-4.00
2,2015/03,10.00,30.0,-4.00
3,2015/04,10.00,30.0,-4.00
4,2015/05,10.20,29.6,-3.20
...,...,...,...,...
119,2024/12,-26.00,-45.0,-13.00
120,2025/01,-28.75,-49.0,-14.75
121,2025/02,-31.50,-53.0,-16.50
122,2025/03,-34.25,-57.0,-18.25


1.2 Data opschonen 
-  perioden wijzigen naar yy/mm 
- 
- datum selecteren 2015-1 t/m 2025-01

# 2. Pompprijzen motorbrandstoffen

https://opendata.cbs.nl/#/CBS/nl/dataset/80416ned/table?ts=1745763454832 

2.1 Data inladen/inzien

2.2 Data opschonen 
- Perioden omzetten naar per maand (first, last, min, mean, avg voor benzine en diesel)
-  perioden wijzigen naar yy/mm 
- datum selecteren 2015-1 t/m 2025-01

2.3 CSV exporteren

# 3. Verkochte wegvoertuigen

https://opendata.cbs.nl/#/CBS/nl/dataset/85898NED/table?ts=1745764299955

3.1 Data inladen/inzien

3.2 Data opschonen 
- Perioden omzetten naar per maand (first, last, min, mean, avg voor benzine en diesel)
-  perioden wijzigen naar yy/mm 
- datum selecteren 2015-1 t/m 2025-01

3.3 CSV exporteren